In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from sklearn.model_selection import train_test_split

NOTEBOOK_DIR = os.getcwd()
ROOT_DIR = os.path.dirname(NOTEBOOK_DIR)
sys.path.append(ROOT_DIR)
from models.attention_mil import build_model
from load_data import get_patches_path, data_generate_resize

DATA_DIR = os.path.join(ROOT_DIR, "data", "colon_ca")
PATCH_WIDTH = 27
PATCH_SHAPE = (PATCH_WIDTH, PATCH_WIDTH, 3)
BATCH_SIZE = 1

CHANGED_SHAPE = (32, 32, 3)

In [ ]:
positive_bag_names = set(os.listdir(os.path.join(DATA_DIR, "1")))
negative_bag_names = set(os.listdir(os.path.join(DATA_DIR, "0")))

bag_names = positive_bag_names | negative_bag_names

patch_dir = os.path.join(DATA_DIR, "patch")
if not os.path.exists(patch_dir):
    os.mkdir(patch_dir)

    for bag_name in bag_names:
        shutil.copy(os.path.join(DATA_DIR, "1", bag_name), os.path.join(patch_dir, bag_name))

In [ ]:
labels = [1 if bag_name in positive_bag_names else 0 for bag_name in bag_names]
patch_bags = get_patches_path(patch_dir)

(
    train_bag_names,
    val_bag_names,
    train_y,
    val_y,
    train_bags,
    val_bags,
) = train_test_split(
    list(bag_names), labels, patch_bags
)

In [ ]:
train_dataset = tf.data.Dataset.from_generator(
        generator=data_generate_resize,
        output_types=(tf.float32, tf.float32),
        output_shapes=(
            tf.TensorShape([None, *CHANGED_SHAPE]),
            tf.TensorShape([1, 1]),
        ),
        args=(train_bag_names, train_y, train_bags),
    )

val_dataset = tf.data.Dataset.from_generator(
    generator=data_generate_resize,
    output_types=(tf.float32, tf.float32),
    output_shapes=(
        tf.TensorShape([None, *CHANGED_SHAPE]),
        tf.TensorShape([1, 1]),
    ),
    args=(val_bag_names, val_y, val_bags),
)

In [ ]:
base_model = tf.keras.applications.vgg16.VGG16(
    input_shape=CHANGED_SHAPE, 
    include_top=False, 
    weights="imagenet", 
)
model = build_model(input_dim=CHANGED_SHAPE, base_model=base_model)
print(model.input_shape, model.output_shape)

In [ ]:
os.makedirs("check_points", exist_ok=True)
model_name = (
        "check_points/"
        + "acc({accuracy:.4f})"
        + "epoch({epoch})"
        + "val_loss({val_loss:.4f}).hd5"
    )

check_point = tf.keras.callbacks.ModelCheckpoint(
    model_name,
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode="min",
)
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=5
)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.2,  # (=decay)
    verbose=True,
)
callbacks = [check_point, early_stopping, reduce_lr]

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=["accuracy"],
)

model.fit(
    train_dataset.repeat(),
    validation_data=val_dataset.repeat(),
    callbacks=callbacks,
    epochs=100,
    steps_per_epoch=int(len(train_bag_names) / BATCH_SIZE),
    validation_steps=int(len(val_bag_names) / BATCH_SIZE),
)